In [1]:
import numpy as np
import pandas as pd
import os
import glob
from pathlib import Path
import unicodedata
import re
import nltk
from nltk import tokenize
import csv

In [ ]:
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word.isalpha()]
    return doc

In [ ]:
df = pd.read_csv('../document similarity/document_similarity_hatt.csv')

In [ ]:
df = pd.read_csv('../GDELT_2017/unrest_gdelt_2017_all.csv')

In [ ]:
df = df.sample(100)

In [ ]:
data = df.iloc[1].content

In [ ]:
# Function for removing unnecessary characters.
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", " ", string)
    string = re.sub(r"\'", " ", string)
    string = re.sub(r"\"", " ", string)
    string = re.sub(r"\n", " ", string)
    string = re.sub(r"=", " ", string)
    string = re.sub(r"-", " ", string)
    string = re.sub(r"/", " ", string)
    return string.strip()

In [ ]:
test_data = unicodedata.normalize("NFKD", data)
test_data = clean_str(test_data)

In [ ]:
import numpy as np
from gensim import corpora
from gensim.models import TfidfModel
from gensim.models import LsiModel
from gensim.similarities import MatrixSimilarity
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
word2vec_model = KeyedVectors.load_word2vec_format("../GoogleNews-vectors-negative300.bin", binary=True)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from gensim.similarities import WmdSimilarity
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from numpy import dot
from numpy.linalg import norm

In [ ]:
# Google Sentence Encoder

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")

In [ ]:
def run_gse_benchmark(sentence1, sentence2):
    sts_input1 = tf.placeholder(tf.string, shape=(None))
    sts_input2 = tf.placeholder(tf.string, shape=(None))

    sts_encode1 = tf.nn.l2_normalize(embed(sts_input1))
    sts_encode2 = tf.nn.l2_normalize(embed(sts_input2))
        
    sim_scores = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
    
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
      
        [gse_sims] = session.run(
            [sim_scores],
            feed_dict={
                sts_input1: sentence1,
                sts_input2: sentence2
            })
    return gse_sims

In [ ]:
test_data = 'Anna Hazare on fast unto death demanding Jan Lokpal Bill  Veteran Gandhian, Mr Anna Hazare, began a fast unto death in the Capital on Tuesday demanding enactment of a comprehensive Jan Lokpal Bill to check corruption. Ignoring an appeal by the Prime Minister, the 72 year old social activist began the Prime Minister day by paying tribute to Mahatma Gandhi at Rajghat. the Prime Minister was joined by activists such as Swami Agnivesh, former IPS officer and activist Ms Kiran Bedi and Magsaysay award winner Mr Sandeep Pandey. “I will observe fast unto death till the Government agrees to form a joint committee comprising 50 per cent officials and the remaining citizens and intellectuals to draft the Jan Lokpal Bill,” Mr Hazare said at Rajghat. On the way to the fast site at Jantar Mantar, Mr Hazare s open jeep was greeted by tricolour waving supporters, including school children. On Monday night, the Prime Minister s Office had said that the Prime Minister s had enormous respect for Mr Hazare and the Prime Minister s mission. “the Prime Minister s says we trust you (Mr Hazare), we respect you. But, then why did the Prime Minister s not sit with we even once after the meeting last month,” said Mr Hazare. Mr Hazare said Mr Hazare was disappointed after the Prime Minister s had, in the last meeting with social activists over the proposed law to tackle corruption, rejected social activists over the proposed law demand for a joint committee. “If the Government alone drafts this Bill, it will be autocratic not democratic,” Mr Hazare said. Mr Hazare lamented that views of eminent persons such as Justice (Retd) Mr Santosh Hegde, senior lawyer Mr Prashant Bhushan and Swami Agnivesh “were not considered important by the Government alone. Meanwhile, reports said that during ‘Gudi Padwa  celebrations in his village Ralegan Siddhi in Ahmednagar district, Maharashtra, the villagers expressed the villagers support by dressing the villagers ‘gudis  in black clothes as a mark of protest. Activists of the ‘India Against Corruption  movement, too, began a fast at Azad Maidan in Mumbai in support of Mr Hazare and a rally of around 100 cars and bikes was organised from Shivaji Park in Central Mumbai to Azad Maidan.'

In [ ]:
import gensim

from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="../glove.840B.300d.txt", word2vec_output_file="../gensim_glove_vectors.txt")
glove_model = KeyedVectors.load_word2vec_format("../gensim_glove_vectors.txt", binary=False)
word2vec_model = KeyedVectors.load_word2vec_format("../GoogleNews-vectors-negative300.bin", binary=True)

In [ ]:
#Word2vec and glove averages.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import math
import nltk

STOP = set(nltk.corpus.stopwords.words("english"))

def run_avg_benchmark(sentences1, sentences2, model=None, use_stoplist=False,doc_freqs = None): 
    
    sims = []
    for (sent1, sent2) in zip(sentences1, sentences2):
    
        tokens1 = [t.lower() for t in nltk.word_tokenize(sent1)]
        #tokens1 = [t for t in tokens1 if t not in STOP]
        
        tokens2 = [t.lower() for t in nltk.word_tokenize(sent2)]
        #tokens2 = [t for t in tokens2 if t not in STOP]
        
        if len(tokens1) == 0 or len(tokens2) == 0:
            sims.append(0)
            continue
        
        tokfreqs1 = Counter(tokens1)
        tokfreqs2 = Counter(tokens2)
        
        weights1 = [tokfreqs1[token] * math.log(N/(doc_freqs.get(token, 0)+1)) 
                    for token in tokfreqs1] if doc_freqs else None
        weights2 = [tokfreqs2[token] * math.log(N/(doc_freqs.get(token, 0)+1)) 
                    for token in tokfreqs2] if doc_freqs else None
        
        emb1 = []
        emb2 = []
        for token in tokfreqs1:
            try:
                emb1.append(model[token])
            except:
                continue;
                
        for token in tokfreqs2:
            try:
                emb2.append(model[token])
            except:
                continue;
                
        embedding1 = np.average(emb1, axis=0, weights=weights1).reshape(1, -1)
        embedding2 = np.average(emb2, axis=0, weights=weights2).reshape(1, -1)

        sim = cosine_similarity(embedding1, embedding2)[0][0]
        sims.append(sim)

    return sims

In [ ]:
# WMD

In [ ]:
def run_wmd_benchmark(sentences1, sentences2, model, use_stoplist=False):
    
    sims = []
    for (sent1, sent2) in zip(sentences1, sentences2):
    
        tokens1 = [t.lower() for t in nltk.word_tokenize(sent1)]
        #tokens1 = [t for t in tokens1 if t not in STOP]
        
        tokens2 = [t.lower() for t in nltk.word_tokenize(sent2)]
        #tokens2 = [t for t in tokens2 if t not in STOP]
        
        model.init_sims(replace=True)
        sims_wmd = WmdSimilarity([tokens1],model)
        sw = sims_wmd[tokens2]
        sims.append(sw)
        
    return sims

In [ ]:
#SIF

In [ ]:
from numpy import dot
from numpy.linalg import norm

In [ ]:
from sklearn.decomposition import TruncatedSVD

def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX


def run_sif_benchmark(sentences1, sentences2, model, freqs={}, use_stoplist=False, a=0.001): 
    total_freq = sum(freqs.values())
    
    embeddings = []
    
    # SIF requires us to first collect all sentence embeddings and then perform 
    # common component analysis.
    for (sent1, sent2) in zip(sentences1, sentences2): 
        
        tokens1 = [t.lower() for t in nltk.word_tokenize(sent1)]
        tokens2 = [t.lower() for t in nltk.word_tokenize(sent2)]
        
        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]
        
        weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
        weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
        embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
        embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)
        
        embeddings.append(embedding1)
        embeddings.append(embedding2)
        
    embeddings = remove_first_principal_component(np.array(embeddings))

    cos_sim = dot(embeddings[0], embeddings[1])/(norm(embeddings[0])*norm(embeddings[1]))

    return cos_sim

In [2]:
#Infer sent
import torch

In [3]:
from models import InferSent
model_version = 1
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

In [4]:
# Keep it on CPU or put it on GPU
use_cuda = False
model = model.cuda() if use_cuda else model

In [5]:
# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
W2V_PATH = 'dataset/GloVe/glove.840B.300d.txt' #if model_version == 1 else '../dataset/fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

In [6]:
# Load embeddings of K most frequent words
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [7]:
sentence1 = "A protest occurred in Nebraska between two groups."
sentence2 = "A protest occured between two groups in Lincoln, Nebraska."

In [8]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [9]:
cosine(model.encode([sentence1],tokenize=True)[0], model.encode([sentence2],tokenize=True)[0])

0.9106413

In [ ]:
sims_infer

In [ ]:
sims = run_gse_benchmark([sentence1],[sentence2])

In [ ]:
sims

In [ ]:
sims_avg_w2v = run_avg_benchmark([sentence1],[sentence2], word2vec_model)

In [ ]:
sims_avg_glove = run_avg_benchmark([sentence1],[sentence2], glove_model)

In [ ]:
sims_wmd_w2v = run_wmd_benchmark([sentence1],[sentence2], word2vec_model)

In [ ]:
sims_wmd_glove = run_wmd_benchmark([sentence1],[sentence2], glove_model)

In [ ]:
def read_tsv(f):
    frequencies = {}
    with open(f) as tsv:
        tsv_reader = csv.reader(tsv, delimiter="\t")
        for row in tsv_reader: 
            frequencies[row[0]] = int(row[1])
        
    return frequencies
        
frequencies = read_tsv('frequencies.tsv')

In [ ]:
#sims_sif_w2v = run_sif_benchmark(["The dog ate the cat"],["The cat was eaten by a dog"] ,word2vec_model,freqs=frequencies)

In [ ]:
#sims_sif_glove = run_sif_benchmark(["Nebraska"],["United States"], glove_model,freqs=frequencies)

In [ ]:
#sims_sif_w2v

In [ ]:
all_sims = []
bottom_sims = []
for ind,row in df.iterrows():
    print(ind)
    sim_doc = []
    bottom_doc = []
    test_data = unicodedata.normalize("NFKD", row['content'])
    test_data = clean_str(test_data)
    sents = nltk.sent_tokenize(test_data)
    title = row['title']
    if len(sents)>5:
        for s in sents[0:5]:
            print(s)
            sims = run_gse_benchmark([title],[s])
            sim_doc.append(sims[0])
        all_sims.append(sim_doc)
        for s in sents[-5:]:
            print(s)
            sims = run_gse_benchmark([title],[s])
            bottom_doc.append(sims[0])
        bottom_sims.append(bottom_doc)
    if (ind+1)%100 == 0:
        t_sims = np.array(all_sims)
        b_sims = np.array(bottoms_sims)
        np.save('top_sim_measures'+str(ind)+'.npy',t_sims)
        np.save('bottom_sim_measures'+str(ind)+'.npy',t_sims)

In [ ]:
df = pd.read_csv('doc_sims.csv')

In [ ]:
sentence1 = df.iloc[0].content
sentence1 = clean_str(sentence1)
sentence2 = df.iloc[1].content
sentence2 = clean_str(sentence2)

In [ ]:
sims = run_gse_benchmark([sentence1],[sentence2])

In [ ]:
i=5;
while(i<=8):
    print(i)
    sentence1 = clean_str(df.iloc[i].content)
    j=i+1;
    while(j<=8):
        print("----");
        print(j);
        sentence2 = clean_str(df.iloc[j].content)
        sims = run_gse_benchmark([sentence1],[sentence2])
        print(sims)
        j=j+1;

In [ ]:
sent_nos = {'Ahmednagar': [11],
 'Azad Maidan': [12],
 'Central Mumbai': [12],
 'Jantar Mantar': [4],
 'Maharashtra': [11],
 'Mahatma Gandhi': [1],
 'Monday night': [5],
 'Mumbai': [12],
 'Rajghat': [1, 3],
 'Ralegan Siddhi': [11],
 'Shivaji Park': [12]}

In [ ]:
sent_nos = {'Anna Hazare': {0, 3, 4, 5, 6, 7, 8, 9, 10, 12},
 'Jan Lokpal Bill Veteran Gandhian': {0, 3, 9},
 'Kiran Bedi': {2},
 'Mahatma Gandhi': {1},
 'Prashant Bhushan': {10},
 'Prime Minister s Office': {5},
 'Sandeep Pandey': {2},
 'Santosh Hegde': {10},
 'Swami Agnivesh': {2, 10},
 '‘ India Against Corruption': {12}}

In [ ]:
text_sent = nltk.sent_tokenize(test_data)
sent_scores = {k:-1 for k in sent_nos.keys()}

In [ ]:
title = 'Anna Hazare on fast unto death demanding Jan Lokpal Bill'

In [ ]:
for s in sent_nos.keys():
    l = sent_nos[s]
    sim = []
    for x in l:
        sims = run_gse_benchmark([title],[text_sent[x]])
        sim.append(sims)
    sent_scores[s] = sim

In [ ]:
sum(sent_scores['Jan Lokpal Bill Veteran Gandhian'])/len(sent_scores['Jan Lokpal Bill Veteran Gandhian'])

In [ ]:
sent_scores